In [1]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Import Dataset
import pandas as pd

df = pd.read_csv('Bert-629-with-index.csv');
df.head(15)

,paragraph,Index
0,BERT <REF> is one of a series of pre-trained n...,0
1,Question <REF> Candidate Paragraph BERT : BERT...,1
2,DecAtt + Doc Reader <REF> 31.4 BERT 50.2 BERT...,2
3,"Specifically, we use SQuAD 1.1 <REF>.",3
4,"Specifically, we use SQuAD 1.1 <REF>.",4
5,"In recent years, deep pre-training approaches ...",5
6,"In recent years, transfer learning has achieve...",6
7,MKDM is implemented from BERT <REF>.,7
8,"In recent work, <REF> show that pretrained mod...",8
9,Inspired by the superiority of Transformer <RE...,9


In [7]:
# Convert to list
data = df.paragraph.values.tolist()

# Remove Emails and <REF>
data = [re.sub('\S*@\S*\s?<REF>', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]


pprint(data[:1])

['BERT <REF> is one of a series of pre-trained neural models that can be fine '
 'tuned to provide state-of-theart results in NLP including on the SQuAD and '
 'NQ tasks that align with our MRC based QA.']


<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-7-bcb88f151c44>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?<REF>', '', sent) for sent in data]
<ipython-input-7-bcb88f151c44>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['bert', 'ref', 'is', 'one', 'of', 'series', 'of', 'pre', 'trained', 'neural', 'models', 'that', 'can', 'be', 'fine', 'tuned', 'to', 'provide', 'state', 'of', 'theart', 'results', 'in', 'nlp', 'including', 'on', 'the', 'squad', 'and', 'nq', 'tasks', 'that', 'align', 'with', 'our', 'mrc', 'based', 'qa']]


In [9]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['bert ref be series pre train neural model can be fine tune provide state theart result nlp include squad nq task align mrc base', 'question ref candidate paragraph bert bert qa base layer hide dimension attention head paramet transformer network large layer hide dimension attention head paramet transformer network model']


In [10]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [21]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_topics=10,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=10, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [22]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=20, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=20, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [23]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -53562.51178872012
Perplexity:  143.11209282594322
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'n_topics': 10,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [24]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [25]:
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_topics': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_topics': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -20025.458886164863
Model Perplexity:  134.94795798413531


#Dominanat topic
To classify a document as belonging to a particular topic, a logical approach is to see which topic has the highest contribution to that document and assign it.

greened out all major topics in a document and assigned the most dominant topic in its own column.

In [28]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.01,0.01,0.01,0.01,0.01,0.01,0.16,0.8,0.01,0.01,7
Doc1,0.01,0.6,0.01,0.01,0.24,0.12,0.01,0.01,0.01,0.01,1
Doc2,0.01,0.01,0.01,0.01,0.23,0.01,0.31,0.01,0.01,0.36,9
Doc3,0.55,0.02,0.02,0.02,0.02,0.02,0.29,0.02,0.02,0.02,0
Doc4,0.55,0.02,0.02,0.02,0.02,0.02,0.29,0.02,0.02,0.02,0
Doc5,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.91,0.01,0.01,7
Doc6,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.91,0.01,0.01,7
Doc7,0.03,0.03,0.03,0.03,0.7,0.03,0.03,0.03,0.03,0.03,4
Doc8,0.01,0.01,0.01,0.01,0.45,0.01,0.24,0.01,0.25,0.01,4
Doc9,0.22,0.18,0.01,0.01,0.27,0.01,0.01,0.08,0.21,0.01,4


In [29]:
#Review topics distribution across documents
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,7,231
1,4,209
2,5,130
3,0,128
4,2,72
5,1,57
6,9,50
7,6,48
8,3,39
9,8,37


In [30]:
#visualize the LDA model with pyLDAvis?
#The pyLDAvis offers the best visualization to view the topics-keywords distribution.

#A good topic model will have non-overlapping, fairly big sized blobs for each topic. This seems to be the case here. So, we are good.

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3     -169.940063  -99.342064       1        1  14.948701
0      -50.139137  -26.852158       2        1  13.507425
5      -45.704956  113.090271       3        1  13.425670
2      156.793930 -147.892059       4        1  10.637171
4     -159.486572   37.086742       5        1  10.123688
7      -65.489410 -165.081390       6        1   9.364552
8      167.257156  -11.461771       7        1   8.665378
9       43.013332 -223.894928       8        1   7.658154
6       47.459114  -83.935799       9        1   6.026358
1       62.816349   54.302124      10        1   5.642904, topic_info=    Category        Freq            Term       Total  loglift  logprob
241  Default  214.000000            word  214.000000  30.0000  30.0000
106  Default  328.000000        language  328.000000  29.0000  29.0000
133  Default   81.000000         network   81.000000  28.0000  28.0000
134  Default   80.000000          neural   80.000000  27.0000  27.0000
222  Default  183.000000     transformer  183.000000  26.0000  26.0000
60   Default  115.000000       embedding  115.000000  25.0000  25.0000
108  Default   80.000000           layer   80.000000  24.0000  24.0000
212  Default  311.000000            task  311.000000  23.0000  23.0000
131  Default   93.000000         natural   93.000000  22.0000  22.0000
18   Default  447.000000            bert  447.000000  21.0000  21.0000
12   Default   78.000000       attention   78.000000  20.0000  20.0000
179  Default  214.000000  representation  214.000000  19.0000  19.0000
125  Default  557.000000           model  557.000000  18.0000  18.0000
217  Default   58.000000           token   58.000000  17.0000  17.0000
14   Default  131.000000            base  131.000000  16.0000  16.0000
168  Default   75.000000          recent   75.000000  15.0000  15.0000
189  Default  121.000000        sentence  121.000000  14.0000  14.0000
58   Default   73.000000            elmo   73.000000  13.0000  13.0000
137  Default  123.000000             nlp  123.000000  12.0000  12.0000
165  Default   61.000000        question   61.000000  11.0000  11.0000
63   Default  110.000000         encoder  110.000000  10.0000  10.0000
169  Default   76.000000        recently   76.000000   9.0000   9.0000
110  Default   84.000000           learn   84.000000   8.0000   8.0000
19   Default   77.000000   bidirectional   77.000000   7.0000   7.0000
156  Default   56.000000       pretraine   56.000000   6.0000   6.0000
160  Default   54.000000      processing   54.000000   5.0000   5.0000
11   Default   84.000000             art   84.000000   4.0000   4.0000
205  Default   90.000000           state   90.000000   3.0000   3.0000
172  Default  969.000000             ref  969.000000   2.0000   2.0000
10   Default   65.000000    architecture   65.000000   1.0000   1.0000
..       ...         ...             ...         ...      ...      ...
128  Topic10    7.490541             mrc   13.082028   2.3172  -4.3981
101  Topic10   13.338246       introduce   32.595520   1.9812  -3.8211
162  Topic10   23.660327         propose   65.178957   1.8614  -3.2479
65   Topic10    5.058295         english   13.939409   1.8611  -4.7907
169  Topic10   27.410914        recently   76.120552   1.8534  -3.1008
64   Topic10    4.676420             end   15.899583   1.6510  -4.8692
95   Topic10   16.511350         include   56.356025   1.6471  -3.6076
46   Topic10   16.913662            deep   58.548671   1.6330  -3.5836
233  Topic10    6.629113         various   25.714992   1.5192  -4.5202
92   Topic10    5.407130           image   21.097188   1.5134  -4.7240
23   Topic10    6.476306         capture   26.598971   1.4620  -4.5435
191  Topic10   11.116486        sequence   51.420883   1.3432  -4.0033
32   Topic10    2.792354        computer   13.230184   1.3192  -5.3848
81   Topic10    2.149238            form   10.617886   1.2773  -5.

In [32]:
#Get top 10 keywords each topic
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=10)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,word,ref,embedding,use,representation,bert,sentence,vector,model,embed
Topic 1,network,neural,ref,model,recently,propose,deep,include,cnn,problem
Topic 2,model,ref,bert,language,elmo,use,pretraine,representation,large,recent
Topic 3,ref,model,language,bert,train,transformer,representation,encoder,pre,bidirectional
Topic 4,ref,bert,model,base,fine,method,tune,approach,good,large
Topic 5,language,task,ref,natural,processing,nlp,model,art,state,recently
Topic 6,task,ref,representation,recent,learn,nlp,learning,transfer,sentence,work
Topic 7,ref,question,text,model,state,task,answer,art,bert,use
Topic 8,ref,layer,transformer,attention,model,bert,base,architecture,use,self
Topic 9,ref,token,sentence,datum,model,task,train,follow,training,predict


In [33]:
#predict the topics for a new piece of text
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["We implement a variant of the word-by-word attention model (Rockta ̈schel et al., 2016) using Tensorflow for this task, where we stack two addi- tional bidirectional RNNs upon the final sequence representation and incorporate character embed- ding for word-level representation. The pretrained GloVe (Pennington et al., 2014) word vectors are used to initialize word embedding. We also inte- grate the base BERT (Devlin et al., 2018) to im- prove contextual modeling"]
topic, prob_scores = predict_topic(text = mytext)
print(topic)


['word', 'ref', 'embedding', 'use', 'representation', 'bert', 'sentence', 'vector', 'model', 'embed']


In [34]:
lda_output.shape

(1001, 10)

In [35]:
df['Topic'] = lda_output.argmax(axis=1)

In [36]:
df.head(1000)

,paragraph,Index,Topic
0,BERT <REF> is one of a series of pre-trained n...,0,7
1,Question <REF> Candidate Paragraph BERT : BERT...,1,1
2,DecAtt + Doc Reader <REF> 31.4 BERT 50.2 BERT...,2,9
3,"Specifically, we use SQuAD 1.1 <REF>.",3,0
4,"Specifically, we use SQuAD 1.1 <REF>.",4,0
5,"In recent years, deep pre-training approaches ...",5,7
6,"In recent years, transfer learning has achieve...",6,7
7,MKDM is implemented from BERT <REF>.,7,4
8,"In recent work, <REF> show that pretrained mod...",8,4
9,Inspired by the superiority of Transformer <RE...,9,4
